In [9]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [10]:
## Main dates (4 набор дат)
date_launch = Time("2026-09-03", scale="utc").tdb
date_flyby_Venus = Time("2027-03-21", scale="utc").tdb
date_flyby_Earth = Time("2029-04-17", scale="utc").tdb
date_arrival = Time("2034-05-15", scale="utc").tdb

In [11]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [12]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [13]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_ven)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_ven, ss_earth_2)
ss_trans_2, ss_target_2 = ss_ven.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_earth_2, ss_jup)
ss_trans_3, ss_target_3 = ss_earth_2.apply_maneuver(man_lambert_3, intermediate=True)

In [14]:
#4
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=90 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=285 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=203 * u.deg), color="black", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [15]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=90 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [16]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=285 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [17]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=203 * u.deg), color="black", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=2 * u.km)

In [18]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [19]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

141748702.441818 km
-47717751.503790826 km
-20670187.206359386 km

[ 9.64775029 25.60369115 11.09830404] km / s


In [20]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

-2956102.566331664 km
-99827883.96427827 km
-44713590.997041464 km

[34.78340309 -0.16975636 -2.27676163] km / s


In [21]:
# Фазовые координаты Земли_2

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

-133785603.35315704 km
-62378793.037437625 km
-27033922.31992602 km

[ 12.98488456 -24.48139493 -10.61115695] km / s


In [22]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

739606192.0652983 km
-45846888.64090448 km
-37650285.93692198 km

[ 0.84098329 12.54265487  5.35555869] km / s


In [23]:
nh_tof_1 = date_flyby_Venus - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_Ven), = iod.lambert(Sun.k, r_earth, r_ven, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=199.0000000344785>

In [24]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 5.82572527 km / s>

In [25]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [ 2.41396303, -4.98474195, -1.80671111] km / s>

In [26]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end = (norm(vk_Ven - v_ven)).to(u.km / u.s)
V_inf_end

<Quantity 9.97764261 km / s>

In [27]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_vec = (vk_Ven - v_ven).to(u.km / u.s)
V_inf_end_vec

<Quantity [-0.03487272, -9.7740382 , -2.00507189] km / s>

In [28]:
nh_tof_2 = date_flyby_Earth - date_flyby_Venus # время полета Венера - Земля2

(v0_Ven, vk_Ea2), = iod.lambert(Sun.k, r_ven, r_earth_2, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=758.0000000001445>

In [29]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_n = (norm(v0_Ven - v_ven)).to(u.km / u.s)
V_inf_end_n

<Quantity 9.76561115 km / s>

In [30]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_n_vec = (v0_Ven - v_ven).to(u.km / u.s)
V_inf_end_n_vec

<Quantity [8.68976603, 3.03522107, 3.26229375] km / s>

In [31]:
# Разница гиперболического избытка скорости при подлете и отлете 

V_gip_diff_1 = V_inf_end - V_inf_end_n 
V_gip_diff_1

<Quantity 0.21203146 km / s>

In [32]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2 = (norm(vk_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2

<Quantity 16.1852979 km / s>

In [33]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_st_2_vec = (vk_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_st_2_vec

<Quantity [15.20424179,  5.24942877,  1.79955475] km / s>

In [34]:
nh_tof_3 = date_arrival - date_flyby_Earth # время полета Земля2 - Юпитер

(V0_Ea2, Vk_Jup), = iod.lambert(Sun.k, r_earth_2, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=1853.999999996233>

In [35]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_3_k_0 = (norm(V0_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_3_k_0

<Quantity 16.24758649 km / s>

In [36]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_3_k_0_vec = (V0_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_3_k_0_vec

<Quantity [-10.62589064, -10.59216214,  -6.23543231] km / s>

In [37]:
# Разница гиперболического избытка скорости при подлете и отлете 

V_gip_diff_2 = V_inf_end_2 - V_inf_3_k_0
V_gip_diff_2

<Quantity -0.06228858 km / s>

In [38]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 9.72978957 km / s>

In [39]:
V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-7.75490896, -5.57074167, -1.87003454] km / s>

In [40]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=2811.000000030856>

In [41]:
scalar_EV = r_earth[0]*r_ven[0] + r_earth[1]*r_ven[1] +r_earth[2]*r_ven[2] 
scalar_VE_2 = r_earth_2[0]*r_ven[0] + r_earth_2[1]*r_ven[1] +r_earth_2[2]*r_ven[2]
scalar_E2J = r_earth_2[0]*r_jup[0] + r_earth_2[1]*r_jup[1] +r_earth_2[2]*r_jup[2]

In [42]:
fi_EV = (scalar_EV/norm(r_earth)/norm(r_ven))
fi_VE2 = (scalar_VE_2/norm(r_earth_2)/norm(r_ven))
fi_E2J = (scalar_E2J/norm(r_earth_2)/norm(r_jup))

In [43]:
np.degrees(np.arccos(fi_EV))

<Quantity 71.40341833 deg>

In [44]:
np.degrees(np.arccos(fi_VE2))

<Quantity 61.51614992 deg>

In [45]:
np.degrees(np.arccos(fi_E2J))

<Quantity 148.62939976 deg>

In [38]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 5.8257253 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 4.55055131 km / s>

In [39]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 9.7297896

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

14.696594699218602
17.983782441612995


<Quantity -3.28718774 km / s>

In [40]:
## параметры участка Землия-Венера 

from math import*

to_rad = pi/180

ist_an_1_EV = 158.6088174 * to_rad
ist_an_2_EV = 86.7732799 * to_rad

exs_EV = (norm(r_ven)/norm(r_earth) - 1) / (cos(ist_an_1_EV) - (norm(r_ven)/norm(r_earth) * cos(ist_an_2_EV)))
p_EV = norm(r_earth)*(1+cos(ist_an_1_EV)*exs_EV)
a_EV = p_EV/(1 - exs_EV*exs_EV)

print (f"Эксцентриситет {exs_EV}")
print (f"Фокальный параметр {p_EV}")
print (f"Большая полуось {a_EV}")

Эксцентриситет 0.2832293637347125
Фокальный параметр 111168633.92809369 km
Большая полуось 120864225.8490634 km


In [41]:
## параметры участка Венера-Землия_2

ist_an_1_VE2 = 343.8882424 * to_rad
ist_an_2_VE2 = 282.4459475 * to_rad

exs_VE2 = (norm(r_earth_2)/norm(r_ven) - 1) / (cos(ist_an_1_VE2) - (norm(r_earth_2)/norm(r_ven) * cos(ist_an_2_VE2)))
p_VE2 = norm(r_ven)*(1+cos(ist_an_1_VE2)*exs_VE2)
a_VE2 = p_VE2/(1 - exs_VE2*exs_VE2)

print (f"Эксцентриситет {exs_VE2}")
print (f"Фокальный параметр {p_VE2}")
print (f"Большая полуось {a_VE2}")

Эксцентриситет 0.5584251423731116
Фокальный параметр 168129309.62065807 km
Большая полуось 244316695.6240368 km


In [42]:
## параметры участка Землия_2 - Юпитер

ist_an_1_E2J = 54.8362819 * to_rad
ist_an_2_E2J = 203.1509914 * to_rad

exs_E2J = (norm(r_jup)/norm(r_earth_2) - 1) / (cos(ist_an_1_E2J) - (norm(r_jup)/norm(r_earth_2) * cos(ist_an_2_E2J)))
p_E2J = norm(r_earth_2)*(1+cos(ist_an_1_E2J)*exs_E2J)
a_E2J = p_E2J/(1 - exs_E2J*exs_E2J)

print (f"Эксцентриситет {exs_E2J}")
print (f"Фокальный параметр {p_E2J}")
print (f"Большая полуось {a_E2J}")

Эксцентриситет 0.7700606275274965
Фокальный параметр 216622173.0266147 km
Большая полуось 532232541.42771256 km
